# Code Explained
First of all we are going to import the required frameworks and tools that we're going to use. We need fastapi in order to make the program RESTful API, uvicorn in oeder to run the program, torch is used to upload the model and using it properly, and some other stuff which I'm going to explain as we go further!

In [ ]:
from fastapi import (FastAPI, 
                     File, 
                     UploadFile, 
                     Request, 
                     HTTPException)
from fastapi.responses import JSONResponse
import torch
from PIL import Image
import io
import uuid
import cv2
import numpy as np
import uvicorn
from fastapi.testclient import TestClient
import pathlib
from pathlib import Path
import sys
import platform


app = FastAPI()

Since my OS is Windows, my program will crash on my docker container if don't set these lines of codes. The reason is docker container has its own ways to deal with paths which is different from what you might see in the Windows operating system.

In [ ]:
sys.path.insert(0, '/objdetection/app/yolov5')
if platform.system() == 'Windows':
    pathlib.PosixPath = pathlib.WindowsPath
else:
    pathlib.WindowsPath = pathlib.PosixPath

CELLPHONEMODEL_PATH = Path(str('/objdetection/app/best.pt'))
SEATBELTMODEL_PATH = Path(str('/objdetection/app/best_seatbelt.pt'))
YOLOV5MODEL = Path('/objdetection/app/yolov5')

- Then we should write an exception handeler for our code. That's why I have defined some configs as a global variables in my code. 
</br>
</br>
- I got the models by torch.hub.load command. I set the device on cpu because the server that is going to run my code, unfortunately doesn't have any GPU available.
</br>
</br>
- running this program on GPU is way faster than CPU. If gpu was available in your server please correct two lines of code below as following:
</br>
</br>
> model_cellphone = torch.hub.load(YOLOV5MODEL, 'custom', path=CELLPHONEMODEL_PATH, force_reload=True, source='local', device='0')
> model_seatbelt = torch.hub.load(YOLOV5MODEL, 'custom', path=SEATBELTMODEL_PATH, force_reload=True, source='local', device='0')

if there are multiple GPU machines available you can pass multiple variables, for instance:
> device = '0, 1, 2'

Before running on gpu make sure that you have cuda installed. You can check if the availability of CUDA like this:
> 1. open cmd
> 2. open the ptrhob interactive shell by writing: python
> </br>
> </br>
> 3. >import torch
> 4. >print(torch.cuda.is_available)

The output should be True. If it's false then something's going wrong!
- Last two lines my code sets the confidence threshold for detecting objects. If the confidence is below this variable the detection result would be false and vice versa. 

In [ ]:
client = TestClient(app)
IMAGEDIR = "fastapi-images/"
image_names = []
allowed_image_types = ["image/jpeg", "image/png", "image/gif"]


#load models from saved '.pt' path 
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
model_cellphone = torch.hub.load(YOLOV5MODEL, 'custom', path=CELLPHONEMODEL_PATH, force_reload=True, source='local', device='cpu') 
model_seatbelt = torch.hub.load(YOLOV5MODEL, 'custom', path=SEATBELTMODEL_PATH, force_reload=True, source='local', device='cpu') 
model_cellphone.conf = 0.25
model_seatbelt.conf = 0.25

This is my exception handeler. There are a few other ways to writing it. you can check it out in the **[fastapi docs](https://devdocs.io/fastapi/)**. 


In [ ]:
class notReceivedException(Exception):
    def __init__(self, name: str):
        self.name = name
@app.exception_handler(notReceivedException)
async def notReceived_exception_handler(request: Request, exc:notReceivedException):
    return JSONResponse(
        status_code=404,
        content={"message": f"Oops! {exc.name} didn't receive!"},
    )

## **"/detect"** POST Method
This method detects both cellphone and seatbelts. 
- At the beginning of the code I have written some exception handelers to see if the file is received, and if the file is the required type. ("jpeg", "png", or "gif") 
- Then I read the files and I rotate one of the images. If you'd ask me why I would say that images saved into "image" variable upside down! That's the reason behind the 180 degree rotation, nothing more!!
- I gave the images to my models! --> results_seatbelt = model_seatbelt(image)
</br>
</br>
The Idea behind the verbose parameter is that I want to be able to return the results in two types! If the verbose is false, then this api will simply return a boolean True or False! However, If you set the query variable (verbose) to true you can get details of your detection. If the detection is "False" then you will get an empty list.
Something like this:
"""</br>
{</br>
  "phone detections": []</br>
}</br>
"""</br>
Conversely, if your detection is "True", then you will get an dictionary which tells you about the **confidence** of the detection, and the **bounding box** which tells you about the location of the detected object in the image in terms of x and y! it will look like something like this:
</br>
"""</br>
{</br>
  "phone detections": [</br>
    {</br>
      "class": "cellphone",</br>
      "confidence": 0.7376633286476135,</br>
      "bbox": [</br>
        70.04457092285156,</br>
        53.837162017822266,</br>
        130.4330291748047,</br>
        104.666748046875</br>
      ]</br>
    }</br>
  ]</br>
}</br>
"""

In [ ]:
@app.post("/detect")
async def detect_objects(file: UploadFile = File(...), verbose:bool = False):
    """
    Detects both cellphones and seatbelts. 
    if you set the query parameter 'verbose' to True it will 
    return in this format rather than true or false:
    _____________________________________________________
    {   
        "class":string (cellphone, seatbelt ...) ,
        "confidence": float (how much confident my model is about the detected object. this parameter is between 0 and 1. 1->totaly confident and 0->not confident at all!),
        "bbox": [x1, y1, x2, y2] (list of four float parameters that shows the location of the detected object in the image)
    }
    _____________________________________________________
    if more than one object is detected then it will return 
    some dictionaries that each of them is associated 
    with one of the detected objects.
    In other words, number of returned dictionaries are the total number of detcted objects!
    """
    #manage input:
    if not file:
        raise notReceived_exception_handler(name = "file")
    if file.content_type not in allowed_image_types:
        raise HTTPException(status_code=400, 
                            detail="Invalid File type. (only jpeg, png or gif are allowed)"
                        )
    # detection = False

    #read images
    image_bytes = await file.read()
    image = Image.open(io.BytesIO(image_bytes))
    image = image.rotate(180)

    #load models
    results_seatbelt = model_seatbelt(image)
    results_cellphone = model_cellphone(image)
    #if verbose is true then we must generate output in a particular format which i have explained above!
    if verbose:
        #generate file name and save it.
        file.filename = f"{uuid.uuid4()}"
        # image.save(f"{IMAGEDIR}{file.filename}.jpg")
        with Image.open(f"{IMAGEDIR}{file.filename}.jpg") as f:
            frame = np.array(f)
            #get the location of the detected object in the image:
            for box in results_cellphone.xyxy[0]: 
                if box[5]==0:
                    # detection = True
                    xB = int(box[2])
                    xA = int(box[0])
                    yB = int(box[3])
                    yA = int(box[1])
                    rect = cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
                    # rect.save(f"{IMAGEDIR}{file.filename}_cellphone.jpg")
                    im = Image.fromarray(rect)
                    # im.save(f"{IMAGEDIR}{file.filename}_cellphone.jpg")
                    # image_names.append(f"{file.filename}_cellphone.jpg")
            for box in results_seatbelt.xyxy[0]: 
                if box[5]==0:
                    xB = int(box[2])
                    xA = int(box[0])
                    yB = int(box[3])
                    yA = int(box[1])
                    #draw a rectabgle around the detected object and save it somewhere:
                    rect = cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
                    im = Image.fromarray(rect)
                    # im.save(f"{IMAGEDIR}{file.filename}_seatbelt.jpg")
            
    seatbelt_detections = []
    phone_detections = []
    for result in results_cellphone.xyxy[0]:
        x1, y1, x2, y2, conf, _ = result.tolist()
        phone_detections.append({
            "class": "cellphone",
            "confidence": conf,
            "bbox": [x1, y1, x2, y2]
        })
        
    for result in results_seatbelt.xyxy[0]:
        x1, y1, x2, y2, conf, _ = result.tolist()
        seatbelt_detections.append({
            "class": "seatbelt",
            "confidence": conf,
            "bbox": [x1, y1, x2, y2]
        })

    return {
        "phone detections": phone_detections if phone_detections and verbose else bool(phone_detections), 
        "seatbelt detections": seatbelt_detections if seatbelt_detections and verbose else bool(seatbelt_detections)
    }

## **"/cellphone"**  & **"/seatbelt"**POST Method
These two work exactly the same with the previous one(detect_objects()). The difference is they only detect the objects that has written in their url path. Verbose qyery oarameter works exactly the same as the previous one.</br>
</br>
**Note**: In all of methods, the default value of verbose is "False". It means that our functions will return bool values until you change it. 

In [ ]:
@app.post("/cellphone")
async def detect_cellphone(file: UploadFile = File(...), verbose:bool=False):
    """
    This function merely returns cellphone detection outputs. 
    The verbose query variable works exactly like previous one. 
    if verbose is true, then instead of True or False it will return 
    a dictionary in the following format:
    _____________________________________________________
    {   
        "class":"cellphone",
        "confidence": float (how much confident my model is about the detected object. this parameter is between 0 and 1. 1->totaly confident and 0->not confident at all!),
        "bbox": [x1, y1, x2, y2] (list of four float parameters that shows the location of the detected object in the image which is cellphone in this case!)
    }
    _____________________________________________________

    """
    if not file:
        raise notReceived_exception_handler(name = "file")
    if file.content_type not in allowed_image_types:
        raise HTTPException(status_code=400, 
                            detail="Invalid File type. (only jpeg, png or gif are allowed)"
                        )
    image_bytes = await file.read()
    image = Image.open(io.BytesIO(image_bytes))
    image = image.rotate(180)
    results_cellphone = model_cellphone(image)
    phone_detections = []
    for result in results_cellphone.xyxy[0]:
        x1, y1, x2, y2, conf, _ = result.tolist()
        phone_detections.append({
            "class": "cellphone",
            "confidence": conf,
            "bbox": [x1, y1, x2, y2]
        })

    return {"phone detections": phone_detections if verbose else bool(phone_detections)}


@app.post("/seatbelt")
async def detect_seatbelt(file:UploadFile=File(...), verbose:bool = False):
    """
    This function merely returns seatbelt detection outputs. 
    The verbose query variable works exactly like previous one. 
    if verbose is true, then instead of True or False it will return 
    a dictionary in the following format:
    _____________________________________________________
    {   
        "class":"seatbelt",
        "confidence": float (how much confident my model is about the detected object. this parameter is between 0 and 1. 1->totaly confident and 0->not confident at all!),
        "bbox": [x1, y1, x2, y2] (list of four float parameters that shows the location of the detected object in the image which is seatbelt in this case!)
    }
    ______________________________________________________
    """
    if not file:
        raise notReceived_exception_handler(name = "file")
    if file.content_type not in allowed_image_types:
        raise HTTPException(status_code=400, 
                            detail="Invalid File type. (only jpeg, png or gif are allowed)"
                        )
    image_bytes = await file.read()
    image = Image.open(io.BytesIO(image_bytes))
    image = image.rotate(180)
    results_seatbelt = model_seatbelt(image)
    seatbelt_detections = []
    for result in results_seatbelt.xyxy[0]:
        x1, y1, x2, y2, conf, _ = result.tolist()
        seatbelt_detections.append({
            "class": "seatbelt",
            "confidence": conf,
            "bbox": [x1, y1, x2, y2]
        })
        
    return {"phone detections": seatbelt_detections if verbose else bool(seatbelt_detections)}


You can ignore the frollowing lines because we are not running this program directly with python command. As you can see in the dockerfile we run this program with: </br>
CMD [ "uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "5000"]</br>
</br>
</br>
**Important Note**: when you run the program with docker it will say you something like this:</br>
> Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)</br>


If you go to your browser and enter the http://0.0.0.0:5000 url, the page will not be loaded. Instead enter http://127.0.0.1:5000 or **localhost:5000**. idk why it works this way but this is what it is!


In [ ]:
if __name__ == "__main__":
   uvicorn.run(app, host="127.0.0.1", port=8000)